In [1]:
import ipywidgets as widgets

In [2]:
def get_cid(data):
    cid = []
    for line in data.splitlines():
        ID = line.split("\t")[6]
        if ID.startswith("C"):
            cid.append(ID)
    return cid

def compute(allBY, allDX):

    DX = get_cid(allDX)
    BY = get_cid(allBY)

    # join the two lists, removing duplicates
    allcompounds = list( dict.fromkeys(DX + BY))

    # transform into sets (to compute set intersection)

    setBY = set(BY)
    setDX = set(DX)
    both = setBY.intersection(setDX)
    onlyBY = setBY.difference(setDX)
    onlyDX = setDX.difference(setBY)
    
    results = {'n1': len(onlyBY), 'n2': len(onlyDX), 'n_common': len(both)}

    text = ["KEGG_ID\tcolor"]

    for compound in allcompounds:
        if compound in both:
            color = 'yellow'
        elif compound in setBY:
            color = 'green'
        else:
            color = 'red'

        text.append( f'{compound}\t{color}' )
    text = '\n'.join(text)
    results['map'] = text
    return results

#### Interface

In [3]:
fname1 = widgets.HTML(value='')
file1_upload = widgets.FileUpload(description='File 1', accept='.annotated', multiple=False)

fname2 = widgets.HTML(value='')
file2_upload = widgets.FileUpload(description='File 2', accept='.annotated', multiple=False)

def on_value_change1(change):
    name = change['new'][0]['name']
    fname1.value = f'<b>{name}</b>'

file1_upload.observe(on_value_change1, names='metadata')

def on_value_change2(change):
    name = change['new'][0]['name']
    fname2.value = f'<b>{name}</b>'

file2_upload.observe(on_value_change2, names='metadata')

hbox1 = widgets.HBox([file1_upload, fname1])
hbox2 = widgets.HBox([file2_upload, fname2])

button = widgets.Button(description="Compute", button_style='primary')
button_reset = widgets.Button(description="Reset")
  
def on_button_clicked(b):
    out.clear_output()
    file_uploads = (file1_upload, file2_upload)
    names = [fu.metadata[0]['name'] for fu in file_uploads]
    with out:
        print("Reading files...")
        for n in names:
            print(f'     {n}')
        print("\nProcessing KeGG IDs...")
    f1_content = file1_upload.data[0].decode('utf-8')
    f2_content = file2_upload.data[0].decode('utf-8')
    results = compute(f1_content, f2_content)
    numbers = [f"   There are {results['n_common']} common KeGG IDs in both files",
               f"   There are {results['n1']} exclusive KeGG IDs in file {names[0]}",
               f"   There are {results['n2']} exclusive KeGG IDs in file {names[1]}"]
    #print_on_html('<br/>'.join(numbers))
    with out:
        print('\n'.join(numbers))
    
        with open("map.txt", "w+") as a:
            a.write(results['map'])
        display(widgets.HTML(value='\nFile <b>map.txt</b> was generated. It maps KeGG IDs to colors'))
        #print('\nFile demo.txt was generated. It maps KeGG IDs to colors')

def on_reset_button_clicked(b):
    out.clear_output()
    file1_upload.metadata.clear()
    file1_upload.data.clear()
    file1_upload.value.clear()
    file1_upload._counter = 0
    file2_upload.metadata.clear()
    file2_upload.data.clear()
    file2_upload.value.clear()
    file2_upload._counter = 0
    
    fname1.value = ''
    fname2.value = ''

    
button.on_click(on_button_clicked)
button_reset.on_click(on_reset_button_clicked)
               
hbuttonbox = widgets.HBox([button, button_reset])

out = widgets.Output()

vbox = widgets.VBox([widgets.HTML(value='<h4>&#9312 Upload two MassTRiX <code>.annotated</code> files:</h4>'),
                     hbox1, hbox2,
                     widgets.HTML(value='<hr/>'),
                     hbuttonbox,
                     out])
vbox